<a href="https://colab.research.google.com/github/sabinbajracharya/spectral-classfication/blob/main/spectogram_generator/ultrasonic-to-spectogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 3.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

from IPython.core.debugger import set_trace

import random
import os
import gc

In [6]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
path = 'https://raw.githubusercontent.com/paweenp/ML-data/master/all_data.xls'
eftrain = pd.read_excel(path, header=None)

path_label = 'https://raw.githubusercontent.com/paweenp/ML-data/master/all_label.xls'
eftrain_label = pd.read_excel(path_label, header=None)

In [8]:
def power_spectral_density_hht_v2(freq, amps, time, freqsol = 33, timesol = 50):
  freqsol = len(freq)
  t0 = time[0]
  t1 = time[-1]

  dt = len(time)

  tw = t1 - t0


  ## Creates 33 columns starting from 0 until 500 
  #
  # [  0.   ,  15.625,  31.25 ,  46.875,  62.5  ,  78.125,  93.75 ,
  # 109.375, 125.   , 140.625, 156.25 , 171.875, 187.5  , 203.125,
  # 218.75 , 234.375, 250.   , 265.625, 281.25 , 296.875, 312.5  ,
  # 328.125, 343.75 , 359.375, 375.   , 390.625, 406.25 , 421.875,
  # 437.5  , 453.125, 468.75 , 484.375, 500.   ]
  bins = np.linspace(0, len(freq), freqsol) #bins = np.linspace(0, 12, freqsol)  # np.logspace(0, 10, freqsol, base=2.0)

  # Assign each item of 2d array to a bin
  # [[1 1 1 ... 1 1 1]
  # [1 1 1 ... 1 1 1]
  # [1 1 1 ... 1 1 1]
  # ...
  # [1 1 1 ... 1 1 1]
  # [0 1 1 ... 1 1 0]
  # [0 1 1 ... 1 1 0]]
  p = np.digitize(freq, bins) #p = np.digitize(freq, 2 ** bins)

  #
  # [ 0.  1.  1. ... 49. 49. 49.]
  t = np.ceil((timesol - 1) * (time - t0) / tw)
  t = t.astype(int)

  # Create 3400x33 array filled with zeros
  hilbert_spectrum = np.zeros([len(time), freqsol]) # (3400, 33)
  

  for i in range(len(time)): # 0 to 3400
      # for j in range(imfs.shape[0] - 1):
    for j in range(len(freq)): # 0 to 7 
      if p[j] >= 0 and p[j] < freqsol:
        hilbert_spectrum[i, p[j]] += amps[j]

  hilbert_spectrum = abs(hilbert_spectrum)

  fig0 = plt.figure(figsize=(5, 5),dpi=50)

  ax = plt.gca()
  ax.set_axis_off()

  plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
  plt.margins(0,0)
  plt.axis('off')

  c = ax.contourf(np.linspace(t0, t1, len(time)), bins, hilbert_spectrum.T) #c = ax.contourf(np.linspace(t0, t1, 3400), timesol, hilbert_spectrum.T) #, colors=('white','lightgray','navy','darkgreen','gold','red'))

  return fig0

In [9]:
PATH = 'gdrive/My Drive/CI/ml/'
PATH_HUMAN = PATH + 'human/'
PATH_EMPTY = PATH + 'empty/'

if not os.path.isdir(PATH_HUMAN):
    os.makedirs(PATH_HUMAN)
    print('Created direcotry:' + PATH_HUMAN)

if not os.path.isdir(PATH_EMPTY):
    os.makedirs(PATH_EMPTY)
    print('Created direcotry:' + PATH_EMPTY)

In [ ]:
for sig_idx in range(eftrain.shape[0]):
  amps = eftrain.iloc[sig_idx].to_numpy()
  freq_len = len(amps)
  freqs = np.linspace(1, freq_len, freq_len)
  datax_t = np.linspace(0, 1, freq_len)

  fig = power_spectral_density_hht_v2(freqs, amps, datax_t)

  label = eftrain_label.iloc[sig_idx].to_numpy()[0]
  if label == 0:
    fig.savefig(PATH_EMPTY +'image'+str(sig_idx)+'.png',dpi=50)
  else:
    fig.savefig(PATH_HUMAN +'image'+str(sig_idx)+'.png',dpi=50)
  
  plt.close(fig)
  plt.cla()
  plt.clf()
  gc.collect()
